In [4]:
from pathlib import Path
import pickle
from PIL import Image

import matplotlib.pyplot as plt

from fastai.vision import ImageDataBunch
from fastai.vision import LabelLists
from fastai.vision import get_transforms

In [5]:
cifar100_stats = (torch.tensor([0.5070751592371323, 0.48654887331495095, 0.4409178433670343]),
 torch.tensor([0.26733428587924035, 0.2564384629170881, 0.2761504713256853]))

In [6]:
path = Path('cifar100')
train_folder = 'train_data'
test_folder = 'test_data'

In [18]:
data = ImageDataBunch.from_folder(path,
                                  train='train_data',
                                  valid='valid_data',
                                  ds_tfms=get_transforms(),
                                  num_workers=0,
                                  bs=1024
                                 ).normalize(cifar100_stats)


In [20]:
%%time
## num_workers=0
next(iter(data.train_dl))

Wall time: 1.64 s


(tensor([[[[-1.7824e+00, -1.7855e+00, -1.7775e+00,  ...,  6.4673e-01,
             6.6715e-01,  6.7661e-01],
           [-1.7823e+00, -1.7877e+00, -1.7855e+00,  ...,  5.2742e-01,
             5.2183e-01,  5.5384e-01],
           [-1.7812e+00, -1.7824e+00, -1.7761e+00,  ...,  3.9472e-01,
             3.8921e-01,  4.5294e-01],
           ...,
           [ 1.1547e+00,  1.1507e+00,  1.1475e+00,  ...,  1.1054e+00,
             1.1314e+00,  1.1089e+00],
           [ 1.1267e+00,  1.1230e+00,  1.1357e+00,  ...,  1.0933e+00,
             1.1254e+00,  1.1063e+00],
           [ 1.1251e+00,  1.1192e+00,  1.1197e+00,  ...,  1.1323e+00,
             1.1332e+00,  1.1681e+00]],
 
          [[-1.7471e+00, -1.7504e+00, -1.7419e+00,  ...,  8.0176e-01,
             8.2318e-01,  8.4670e-01],
           [-1.7471e+00, -1.7528e+00, -1.7505e+00,  ...,  6.9518e-01,
             6.7932e-01,  7.3156e-01],
           [-1.7459e+00, -1.7472e+00, -1.7404e+00,  ...,  5.5979e-01,
             5.4774e-01,  6.2300e-01],


In [22]:
data = ImageDataBunch.from_folder(path,
                                  train='train_data',
                                  valid='valid_data',
                                  ds_tfms=get_transforms(),
                                  num_workers=4,
                                  bs=1024
                                 ).normalize(cifar100_stats)

In [57]:
%%time
## num_workers=4
next(iter(data.train_dl))

Wall time: 15.5 s


In [50]:
import torch
from torch.utils.data import Dataset, DataLoader, IterableDataset
from PIL import Image
import numpy as np

In [69]:
def unpickle(file):
    with open(file, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data

In [71]:
path = Path('cifar100')
train_file = 'train'
test_file = 'test'

In [77]:
class Cifar100Dataset(Dataset):
    def __init__(self, path, folder):
        self.files = [file for directory in (path / folder).ls() for file in directory.ls()]
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        image = torch.tensor(np.array(Image.open(self.files[i]))).permute(2, 0 ,1)
        return image, int(self.files[i].parent.stem)

In [80]:
train_ds = Cifar100Dataset(path, train_folder)

In [85]:
train_dl = DataLoader(train_ds, batch_size=1024, num_workers=4, shuffle=True)

In [86]:
%%time
next(iter(train_dl))

BrokenPipeError: [Errno 32] Broken pipe

In [107]:
def read_files(files):
    images = [torch.tensor(np.array(Image.open(i))) for i in files]
    yield images

In [108]:
class Cifar100Dataset(IterableDataset):
    def __init__(self, path, folder, start, end):
        self.files = [file for directory in (path / folder).ls() for file in directory.ls()]
        self.start = start
        self.end = end
    
    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:
            return read_files(self.files[self.start:self.end])
        else:
            per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end = min(iter_start + per_worker, self.end)
            return iter(read_files(self.files[iter_start:iter_end]))

In [131]:
ds = Cifar100Dataset(path, train_folder, start=0, end=1024)

In [148]:
train_dl = DataLoader(ds, batch_size=1, num_workers=0)

In [149]:
%%time
xx = next(iter(train_dl))

Wall time: 692 ms


In [150]:
len(xx)

1024

In [139]:
train_dl = DataLoader(ds, batch_size=1, num_workers=2)


In [140]:
%%time
xx = next(iter(train_dl))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001F99F5CB9D8>
Traceback (most recent call last):
  File "C:\Users\Divyansh J\Anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 883, in __del__
    self._shutdown_workers()
  File "C:\Users\Divyansh J\Anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py", line 860, in _shutdown_workers
    if self.workers_status[worker_id]:
IndexError: list index out of range


BrokenPipeError: [Errno 32] Broken pipe